# Wages by County Interpolation

## 1. Import important resources

### 1.1 Import Modules

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime

#### 1.1.1 Instantiate geocoder.

In [177]:
geolocator = Nominatim(user_agent='Reverse Geocoder')
location = geolocator.geocode('Naperville, IL')
location
print(location.raw['addresstype'], '-->', location.raw['display_name'])
# print(location.raw)

city --> Naperville, DuPage County, Illinois, United States


#### 1.1.2 Create function that parses the county.

In [11]:
print(parseCountyFromLocation(location.address))

Benton County


### 1.2 Import Data

In [14]:
wagesByCity = pd.read_csv('.\data\wages_level_4.csv')

C:\Users\Dean Akin\AppData\Local\Temp\ipykernel_20016\1653995885.py:1: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  wagesByCity = pd.read_csv('.\data\wages_level_4.csv')


In [15]:
wagesByCity.head(1)

,AREA,AREA_TITLE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,O_GROUP,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,11500,"Anniston-Oxford-Jacksonville, AL",AL,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,total,...,18.2,26.87,36.83,21860,28270,37850,55890,76610,NaN,NaN


## 2. Prepping Data for reverse geocoding

#### 2.1 Retrieve unique city-state pairs.

In [62]:
uniqueCityStatePairs = wagesByCity.AREA_TITLE.unique()
print(uniqueCityStatePairs)

['Anniston-Oxford-Jacksonville, AL' 'Auburn-Opelika, AL'
 'Birmingham-Hoover, AL' 'Daphne-Fairhope-Foley, AL' 'Decatur, AL'
 'Dothan, AL' 'Florence-Muscle Shoals, AL' 'Gadsden, AL' 'Huntsville, AL'
 'Mobile, AL' 'Montgomery, AL' 'Tuscaloosa, AL' 'Anchorage, AK'
 'Fairbanks, AK' 'Flagstaff, AZ' 'Lake Havasu City-Kingman, AZ'
 'Phoenix-Mesa-Scottsdale, AZ' 'Prescott, AZ' 'Sierra Vista-Douglas, AZ'
 'Tucson, AZ' 'Yuma, AZ' 'Bakersfield, CA' 'Chico, CA' 'El Centro, CA'
 'Fresno, CA' 'Hanford-Corcoran, CA' 'Los Angeles-Long Beach-Anaheim, CA'
 'Madera, CA' 'Merced, CA' 'Modesto, CA' 'Napa, CA'
 'Oxnard-Thousand Oaks-Ventura, CA' 'Redding, CA'
 'Riverside-San Bernardino-Ontario, CA'
 'Sacramento--Roseville--Arden-Arcade, CA' 'Salinas, CA'
 'San Diego-Carlsbad, CA' 'San Francisco-Oakland-Hayward, CA'
 'San Jose-Sunnyvale-Santa Clara, CA'
 'San Luis Obispo-Paso Robles-Arroyo Grande, CA'
 'Santa Cruz-Watsonville, CA' 'Santa Maria-Santa Barbara, CA'
 'Santa Rosa, CA' 'Stockton-Lodi, CA' 'Vallejo

#### 2.2 Extract metropolitan areas and/or cross-state cities

In [63]:
uniqueCityStatePairs = [item for item in uniqueCityStatePairs if '-' in item]
print(uniqueCityStatePairs)

['Anniston-Oxford-Jacksonville, AL', 'Auburn-Opelika, AL', 'Birmingham-Hoover, AL', 'Daphne-Fairhope-Foley, AL', 'Florence-Muscle Shoals, AL', 'Lake Havasu City-Kingman, AZ', 'Phoenix-Mesa-Scottsdale, AZ', 'Sierra Vista-Douglas, AZ', 'Hanford-Corcoran, CA', 'Los Angeles-Long Beach-Anaheim, CA', 'Oxnard-Thousand Oaks-Ventura, CA', 'Riverside-San Bernardino-Ontario, CA', 'Sacramento--Roseville--Arden-Arcade, CA', 'San Diego-Carlsbad, CA', 'San Francisco-Oakland-Hayward, CA', 'San Jose-Sunnyvale-Santa Clara, CA', 'San Luis Obispo-Paso Robles-Arroyo Grande, CA', 'Santa Cruz-Watsonville, CA', 'Santa Maria-Santa Barbara, CA', 'Stockton-Lodi, CA', 'Vallejo-Fairfield, CA', 'Visalia-Porterville, CA', 'Denver-Aurora-Lakewood, CO', 'Cape Coral-Fort Myers, FL', 'Crestview-Fort Walton Beach-Destin, FL', 'Deltona-Daytona Beach-Ormond Beach, FL', 'Lakeland-Winter Haven, FL', 'Miami-Fort Lauderdale-West Palm Beach, FL', 'Naples-Immokalee-Marco Island, FL', 'North Port-Sarasota-Bradenton, FL', 'Orlando

#### 2.3 Interpreting the list of cross-state cities
1. Some cities may be census-designated place that are designated as a county or a city that has no county which makes it an independant city which counts itself as it's own county, or its a legitimate city (i.e. see Arlington, VA)
   1. In this case we need to see for each city and state pair what satisfies the former conditions.
2. The position of the concatenated string in both the city and state section of the string are important.
   1. If it has a format of- "city1-city2-..., state", the query would be fore "city1, state" as that is the largest city of that specific metro area.
   2. 